In [3]:
import modules
import importlib
importlib.reload(modules)

# design variables
x = {'capture_width': 10,
    'wec_type': 'point absorber',
    'pen_diameter': 1,
    'pen_height': 0.5 }

# parameters
wec_types = ['attenuator','terminator','point absorber']
capture_width_ratios = [0.5, 0.5, 0.5]
wave_dampings = [0.5, 0.5, 0.5]

p = {'wave_height': 2,
    'wave_period': 5,
    'wave_damping_dict': dict(zip(wec_types, wave_dampings)),
    'capture_width_ratio_dict': dict(zip(wec_types, capture_width_ratios)),
    'wec_unit_cost': 1000,
    'pen_unit_cost': 1000}

J, g, h = modules.sim(x,p)

